## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [22]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.head(10))
data.info()

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод

Каждая строка содержит информацию о семейном положении, трудовой деятельности, доходе и задолженности по кредиту клиента банка. В глаза бросаются следующие проблемы: неверный тип данных в столбце об общем трудовом стаже и в столбце об образованиии знчения внесены в разных регистрах, в столбце 'purpose' имеются значения с одной основой.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [23]:
data.isnull().sum()
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
#data.isnull().sum()

### Вывод

Были клиенты, для которых доступна не вся информация. Пропуски наблюдались  в количественных переменных 'days_employed' и 'total_income'. Пропуски в стаже работы и доходе были заменены на медиану.

### Замена типа данных

In [24]:
#data.info()
data['days_employed'] = data['days_employed'].astype('int')
data.loc[:,'days_employed'] = data.loc[:,'days_employed'].abs()
data.loc[:,'children'] = data.loc[:,'children'].abs()

data['total_income'] = data['total_income'].astype('int')
#print(data.head())

### Вывод

Ранее были обнаружены ошибки в типе данных в столбцах 'days_employed' и 'total_income', они были устранены методом astype() и приведены к типу 'int'. Также знечения в столбцах 'days_employed', 'children' были взяты по модулю, чтобы значения не были отрицательными.

### Обработка дубликатов

In [25]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())



0


### Вывод

В столбце 'education' были дубликаты с разными регистрами, методом str.lower() все значения привели к нижнему регистру. Методом value_counts() нашли частоту встречаемости каждого уникального значения в столбце 'education'. Методом drop_duplicates() удалили дубликаты, содержащиеся в данных.

### Лемматизация

In [26]:

from pymystem3 import Mystem
m = Mystem()
data['purpose'] = data['purpose'].apply(lambda x: ''.join(m.lemmatize(x)))
print(data['purpose'])



0                     покупка жилье\n
1           приобретение автомобиль\n
2                     покупка жилье\n
3        дополнительный образование\n
4                   сыграть свадьба\n
                     ...             
21449              операция с жилье\n
21450           сделка с автомобиль\n
21451                  недвижимость\n
21452    на покупка свой автомобиль\n
21453         на покупка автомобиль\n
Name: purpose, Length: 21454, dtype: object


### Вывод

Импортируем библиотеку pymystem3. С помощью метода apply() и лямда функции применим лемматизацию к столбцу 'purpose' и склеим.

### Категоризация данных

In [27]:
#Категоризация целей получения кредита
def purpose_group(row):
        if  ('жилье' or 'недвижимость')  in row:
            return 'недвижимость'
        elif 'автомобиль' in row:
            return 'автомобиль'
        elif 'образование' in row:
            return 'образование'
        elif  'свадьба' in row:
            return 'свадьба'
data['purpose_category'] = data['purpose'].apply(purpose_group)
print(data['purpose_category'].value_counts())



недвижимость    4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_category, dtype: int64


In [28]:
#Категоризация ежемесячного дохода
#print(data['total_income'].min() , data['total_income'].max(), data['total_income'].median() )
def income_level(x):
    if x < 150000:
        return 'низкий уровень дохода'
    if x <= 300000:
        return 'средний уровень дохода'
    return 'высокий уровень дохода'
data['income_level'] = data['total_income'].apply(lambda x: income_level(x))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилье\n,недвижимость,средний уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиль\n,автомобиль,низкий уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилье\n,недвижимость,низкий уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительный образование\n,образование,средний уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьба\n,свадьба,средний уровень дохода


### Вывод

После лемматизации столбца с целями получения кредита можно приступать к категоризации на 4 категории: недвижимость, автомобиль, образование, свадьба. Для этого была написана функция purpose_group(row) принимающая в качестве аргумента строку и по некоторому ключеву слову определят принадлежность к той или иной категории. Методом apply и lambda-функцией применили к столбцу 'purpose'. Аналогичным образом разделили ежемесячный доход клиентов по уровням дохода. Вызвав минимальный,максимальный и медианный значения определили границы дохода.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
#data[data['children']==20]=2

data['children'] = data['children'].replace(20,2)  #изменение в коде

children_debt_table = data.pivot_table(index=['children'], columns='debt', values='family_status_id', aggfunc='count')

#процент просрочивших кредит:
children_debt_table['percent'] = children_debt_table[1] * 100 / (children_debt_table[0] + children_debt_table[1]) 

print(children_debt_table)

debt            0       1   percent
children                           
0         13028.0  1063.0  7.543822
1          4410.0   445.0  9.165808
2          1926.0   202.0  9.492481
3           303.0    27.0  8.181818
4            37.0     4.0  9.756098
5             9.0     NaN       NaN


### Вывод

Из полученной сводной таблицы видим, что с появлением детей и увеличением их количества желание брать кредиты падает, также падает возможность  погашения задолженности. Это видно из столбца с процентами просрочивших кредит 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
family_status_debt_table = data.pivot_table(index=['family_status'], columns='debt', values='family_status_id', aggfunc='count')

# процент просрочивших кредит
family_status_debt_table['percent'] = family_status_debt_table[1] * 100 / (family_status_debt_table[0] + family_status_debt_table[1])

print(family_status_debt_table.sort_values(by='percent', ascending=False))

debt                       0    1   percent
family_status                              
Не женат / не замужем   2536  274  9.750890
гражданский брак        3763  388  9.347145
женат / замужем        11408  931  7.545182
в разводе               1110   85  7.112971
вдовец / вдова           896   63  6.569343


### Вывод

Из приведенной таблицы видим, что наибольшее количество кредитов берут семейные люди и совместными усилиями возвращают кредит в срок. Не женатых / не замужних должников больше всего, их почти 10%.  Наименьшее количество кредитов у овдовевших людей и минимальная задолженность, они составляют 6.6%.   

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
income_debt_table = data.pivot_table(index=['income_level'], columns='debt', values='income_type', aggfunc='count')

#процент просрочивших кредит
income_debt_table['percent'] = income_debt_table[1] * 100 / (income_debt_table[0] + income_debt_table[1])

print(income_debt_table.sort_values(by='percent'))

debt                        0     1   percent
income_level                                 
высокий уровень дохода   1376   106  7.152497
средний уровень дохода   7082   620  8.049857
низкий уровень дохода   11255  1015  8.272209


### Вывод

Люди с высоким доходом берут меньше всего кредитов и задолженность по кредиту минимальная, что составляет чуть больше 7%. Таким образом, можно сделать вывод, что чем выше доход у клиента, тем выше его вероятность, что кредит будет выплачен в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
purpose_debt_table = data.pivot_table(index=['purpose_category'], columns='debt', values='family_status_id', aggfunc='count')

#процент просрочивших кредит
purpose_debt_table['percent'] = purpose_debt_table[1] * 100 / (purpose_debt_table[0] + purpose_debt_table[1])

print(purpose_debt_table.sort_values(by='percent'))

debt                 0    1   percent
purpose_category                     
недвижимость      4152  308  6.905830
свадьба           2138  186  8.003442
образование       3643  370  9.220035
автомобиль        3903  403  9.359034


### Вывод

Люди больше всего берут кредит на недвижимость, так как самое важное крыша над головой и собственное имущество, также это и часть дохода. Так как, недвижимость является  одним из самых необходимых вещей для жизни, то люди стараются во время выплачивать кредит, чтобы не потерять. На втором месте автомобиль, в современном мире без личного средства передвижения сложно угнаться за ритмом жизни, но без него тоже можно обойтись. И в то же время  сумма ежемесячного платежа выше, чем по среднему необеспеченному кредиту. Поэтому клиенты делают выбор в пользу  жизненно необходимых затрат или меньшей суммы долга.

### Шаг 4. Общий вывод

Таким образом, из проведенного анализа можно сделать вывод, что женатые/замужние пока без детей клиенты охотнее берут кредиты. А возвращать кредит легче совместными усилиями, нежели по одиночке. Наиболее важные цели кредита- это недвижимость, чтобы была своя крыша над головой, когда появятся дети,  и автомобиль, средство личного передвижения. В то же время, люди тратят деньги на первостепенные, наиболее важные для жизни или легкодоступные вещи. Также чем  выше доход у людей, тем выше возвратность кредита в срок.



### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.